In [1]:
import numpy as np
import pandas as pd
from importlib import reload 

import util.performance_metrics as pm
 
reload(pm)

url =  "https://raw.githubusercontent.com/xuda1979/DF/master/sp500_companies/data/constituents.csv"
df_symbols =pd.read_csv('./sp500_companies/data/constituents.csv')

df_symbols.head()
all_symbols = list(df_symbols['Symbol'].unique())
all_symbols.append('SPY')

# Import yfinance
import yfinance as yf  
 
# Get the data for the stock Apple by specifying the stock ticker, start date, and end date
data_all =pd.DataFrame({})
df_ls = []
for symbol in all_symbols:
    
    data = yf.download(symbol)#,'2016-01-01','2018-01-01')
    data['Symbol'] = symbol
    if(len(data)>0):
        df_ls.append(data)
data_all = pd.concat(df_ls)
 

vix=yf.download('^VIX')

symbols = list(data_all['Symbol'].unique())

data_all.reset_index(inplace=True)

data_all.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- NBL: No data found, symbol may be delisted
[*********************100%************

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1970-01-02,6.851563,6.890625,6.843750,6.851563,1.483355,72000.0,MMM
1,1970-01-05,6.859375,6.898438,6.859375,6.890625,1.491813,446400.0,MMM
2,1970-01-06,6.890625,6.960938,6.882813,6.960938,1.507035,176000.0,MMM
3,1970-01-07,6.960938,7.015625,6.945313,7.000000,1.515492,164800.0,MMM
4,1970-01-08,7.000000,7.109375,6.984375,7.093750,1.535788,304000.0,MMM


In [190]:
vix.reset_index(inplace=True)

In [8]:
import os
paths = os.listdir('./short_data/')

In [11]:
df_ls = []
for path in paths:
    if path[:9] =='NPSXshvol':
        df = pd.read_csv(os.path.join('./short_data', path),sep='|')
        df_ls.append(df)
    
df_short = pd.concat(df_ls)
    

In [20]:
df_short['short_rate'] = df_short['SHORT VOLUME']/df_short['TOTAL VOLUME']

In [21]:
df_short_mp = {}
for symbol in set(df_short.SYMBOL):
    mask = df_short.SYMBOL == symbol
    df = df_short[mask].reset_index(drop=True)
    df_short_mp[symbol] = df


In [48]:
for symbol in df_short_mp:
    df_short_mp[symbol]['DATE']= pd.to_datetime(df_short_mp[symbol]['DATE'], format='%Y%m%d')

In [49]:
df_short_mp['AAPL']

,DATE,SYMBOL,SHORT VOLUME,TOTAL VOLUME,MARKET,short_rate
0,2013-01-25,AAPL,5500,28344,X,0.194045
1,2017-05-01,AAPL,194376,319036,X,0.60926
2,2016-07-12,AAPL,113412,244005,X,0.464794
3,2013-01-31,AAPL,400,3200,X,0.125
4,2012-03-22,AAPL,9620,13420,X,0.716841
...,...,...,...,...,...,...
2584,2018-08-27,AAPL,62080,117933,X,0.526401
2585,2017-05-18,AAPL,87032,191282,X,0.454993
2586,2012-10-31,AAPL,1700,4900,X,0.346939
2587,2015-08-10,AAPL,398843,736133,X,0.541808


# Risk Model

In [931]:
def covariance(df, date, symbols, history):
    df_recent = pd.DataFrame({})
    dates = list(df['Date'].unique())
    dates.sort()
    df_recent['Date']= dates[-(history+1):-1]
    for symbol in symbols:
        mask = df['Symbol'] == symbol
 
        if all([date in list(df[mask]['Date'].unique()) for date in dates[-(history+1):-1]]):
            #print(df[mask]['Close'][-(history+1):-1])
            df_recent[symbol] = df[mask]['Close'].to_list()[-(history+1):-1]
    df_return = pd.DataFrame({})
    df_return['Date'] = df_recent['Date']
    for symbol in df_recent.columns:
        if symbol !='Date':
            #print(symbol,df_recent[symbol].div(df_recent[symbol].shift(1)))
            df_return[symbol] = df_recent[symbol].div(df_recent[symbol].shift(1))
    return df_return[1:].set_index('Date').cov() 
 
            
symbols = ['AAPL', 'MMM']
df_cov = covariance(data_training, list(data_training['Date'].unique())[-1], symbols, 10)
cov=df_cov.to_numpy() 
print(cov)

[[7.85042934e-04 7.44584306e-06]
 [7.44584306e-06 3.88371563e-04]]


In [988]:
def objective(x):
    l = len(x)
    assert l == cov.shape[0]
    return x.dot(cov.dot(x))

def constraint(x):
    return np.atleast_1d(np.sum(np.abs(x)) - 1.5)

 
bounds=[[0, 1], [1, 2]]
res=optimize.minimize(objective, np.array([0, 0]), constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': True}, bounds=bounds )
res.x    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0006603349124938092
            Iterations: 2
            Function evaluations: 6
            Gradient evaluations: 2


<ipython-input-988-e88c3116b488>:11: OptimizeWarning: Unknown solver options: gtol
  res=optimize.minimize(objective, np.array([0, 0]), constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': True}, bounds=bounds )


array([0.25038093, 1.24961907])

In [1025]:
from scipy.optimize import minimize
def portfolio_optimization(cov, k):        
 
    assert 2*k == cov.shape[0]
    def objective(x):
        return x.dot(cov.dot(x))

    def constraint(x):
        return np.atleast_1d(np.sum(np.abs(x))-1)
    x0 = [1/(2*k)]*k+[-1/(2*k)]*k
    x0 = np.array(x0)
    bounds = []
    for i in range(k):
        bounds.append([1/(4*k),1/k])
    for i in range(k):
        bounds.append([-1/k, -1/(4*k)])
        
    res=optimize.minimize(objective, x0, constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': False}, bounds=bounds )
    s= sum(np.abs(res.x))
    return res.x/s 

In [133]:
data_all[data_all.Symbol == 'MMM'].Close.iloc[-1]

179.00999450683594

In [134]:
data_all

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1970-01-02,6.851563,6.890625,6.843750,6.851563,1.483355,72000.0,MMM
1,1970-01-05,6.859375,6.898438,6.859375,6.890625,1.491813,446400.0,MMM
2,1970-01-06,6.890625,6.960938,6.882813,6.960938,1.507035,176000.0,MMM
3,1970-01-07,6.960938,7.015625,6.945313,7.000000,1.515492,164800.0,MMM
4,1970-01-08,7.000000,7.109375,6.984375,7.093750,1.535788,304000.0,MMM
...,...,...,...,...,...,...,...,...
3770329,2021-02-01,373.720001,377.339996,370.380005,376.230011,376.230011,75817600.0,SPY
3770330,2021-02-02,379.649994,383.220001,376.320007,381.549988,381.549988,64450700.0,SPY
3770331,2021-02-03,382.440002,383.700012,380.480011,381.850006,381.850006,52427100.0,SPY
3770332,2021-02-04,382.959991,386.239990,381.970001,386.190002,386.190002,47142600.0,SPY


# Trade at Open(close-open error as regressor)  

In [150]:
from importlib import reload
import train
import data_processing.data_processing as dp
dp = reload(dp)
train = reload(train)

In [146]:
start_date = '20150101'
end_date = '20190101'
sm_reg_map, sm_reg_map_reverse = train.train_model(data_all, start_date, end_date, symbols,)


In [147]:
sm_reg_map['AAPL'].params

const    0.000142
x1       1.168441
dtype: float64

In [148]:
sm_reg_map_reverse['AAPL'].tvalues

const     0.654053
x1       32.923649
x2       -2.156141
dtype: float64

In [175]:
mask = data_all.Symbol.isin(set(sm_reg_map_reverse))
mask = data_all.Date == '20200102'
sum(data_all[mask].Close < 10)

6

In [174]:
len(sm_reg_map_reverse)

447

In [159]:

sum(data_all[mask].Close <20 & data_all[mask].Symbol.isin(sm_reg_map_reverse))

0

In [151]:
train.back_testing(
    1,
    data_all,
    symbols,
    sm_reg_map,
    sm_reg_map_reverse,
    '20190101',
    None, 
)

last 89.98999786376953
last 49.849998474121094
last 41.66999816894531
last 144.9199981689453
last 115.20999908447266
last 43.09000015258789
last 105.94999694824219
last 62.52000045776367
last 56.02000045776367
last 91.44000244140625
last 38.189998626708984
last 57.029998779296875
last 159.58999633789062
last 87.44000244140625
last 24.959999084472656
last 172.41000366210938
last 11.390000343322754
last 9.760000228881836
last 52.209999084472656
last 21.329999923706055
last 43.41999816894531
last 26.309999465942383
last 183.41000366210938
last 16.84000015258789
last 75.33000183105469
last 95.88999938964844
last 126.5999984741211
last 54.4900016784668
last 264.7300109863281
last 67.80000305175781
last 267.6600036621094
last 27.299999237060547
last 92.6500015258789
last 46.939998626708984
last 97.79000091552734
last 37.88999938964844
last 84.4800033569336
last 84.0
last 84.18000030517578
last 16.389999389648438
last 39.619998931884766
last 17.809999465942383
last 101.12000274658203
last 31.

last 64.43000030517578
last 71.88999938964844
last 102.27999877929688
last 41.034156799316406
last 59.40999984741211
last 48.220001220703125
last 75.13999938964844
last 106.69000244140625
last 53.529998779296875
last 34.369998931884766
last 93.33999633789062
last 65.10841369628906
last 107.88999938964844
last 127.75
last 350.0
last 52.43000030517578
last 28.850000381469727
last 75.62000274658203
last 56.63999938964844
last 38.43000030517578
last 47.029998779296875
last 81.63999938964844
last 72.94000244140625
last 23.700000762939453
last 67.30999755859375
last 25.75
last 194.47000122070312
last 838.47998046875
last 114.77999877929688
last 38.38999938964844
last 64.31999969482422
last 130.13999938964844
last 304.69000244140625
last 20.6299991607666
last 9.289999961853027
last 75.55999755859375
last 250.17999267578125
last 77.26000213623047
last 466.0299987792969
last 115.80999755859375
last 35.27000045776367
last 7.740385055541992
last 90.91000366210938
last 41.33000183105469
last 131.9

last 111.30999755859375
last 64.16000366210938
last 77.9000015258789
last 63.970001220703125
last 65.52999877929688
last 45.70000076293945
last 75.06999969482422
last 28.030000686645508
last 50.97999954223633
last 112.29000091552734
last 57.869998931884766
last 44.91999816894531
last 40.77000045776367
last 38.279998779296875
last 64.2300033569336
last 74.04000091552734
last 117.20999908447266
last 39.7400016784668
last 67.76000213623047
last 69.79000091552734
last 24.700000762939453
last 85.44999694824219
last 45.61000061035156
last 90.70999908447266
last 183.75999450683594
last 71.56999969482422
last 37.75
last 184.77999877929688
last 31.540000915527344
last 287.5
last 52.5
last 46.560001373291016
last 28.450000762939453
last 325.54998779296875
last 1500.280029296875
last 56.52000045776367
last 41.06999969482422
last 68.05999755859375
last 310.8999938964844
last 128.1300048828125
last 23.520000457763672
last 0.03500000014901161
last 109.61000061035156
last 51.040000915527344
last 90.7

last 59.52000045776367
last 15.40999984741211
last 67.44821166992188
last 37.064998626708984
last 23.020000457763672
last 184.67999267578125
last 48.79999923706055
last 247.89999389648438
last 28.68000030517578
last 216.99000549316406
last 151.67999267578125
last 0.07999999821186066
last 26.0
last 37.54999923706055
last 81.75077056884766
last 43.959999084472656
last 39.880001068115234
last 10.6899995803833
last 56.76499938964844
last 32.040000915527344
last 17.09000015258789
last 139.2899932861328
last 302.69000244140625
last 69.01000213623047
last 206.24000549316406
last 65.45999908447266
last 51.369998931884766
last 255.02999877929688
last 49.573333740234375
last 31.34000015258789
last 455.0
last 16.600000381469727
last 47.790000915527344
last 196.60000610351562
last 59.560001373291016
last 110.81999969482422
last 19.0
last 53.72999954223633
last 108.7699966430664
last 44.91999816894531
last 85.37000274658203
last 110.4800033569336
last 157.77000427246094
last 341.82000732421875
last

KeyboardInterrupt: 

# Model deployment

In [91]:
from importlib import reload
import data_processing.data_processing as dp
import train
train = reload(train)
dp = reload(dp)

In [92]:
 

start_date = '20150101'
end_date = '20191231'
end_date = None
sm_reg_map, sm_reg_map_reverse = train.train_model(data_all, start_date, end_date, symbols)

/usr/local/anaconda3/envs/df/lib/python3.8/site-packages/statsmodels/robust/robust_linear_model.py:287: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  warnings.warn('Estimated scale is 0.0 indicating that the most'


In [93]:
dp.save0(sm_reg_map)
dp.save(sm_reg_map_reverse)

 

In [94]:
df=pd.read_csv('params.csv')

In [95]:
df

,symbol,const,x1,x2,tvalue
0,ECL,0.000076,0.907428,-0.126732,-3.514918
1,FLIR,-0.000019,0.955897,-0.122764,-4.732049
2,CCI,0.000188,0.614708,-0.151185,-3.255271
3,GRMN,0.000075,0.942156,-0.195367,-7.531342
4,PLD,0.000350,0.855984,-0.262958,-5.013329
...,...,...,...,...,...
163,NTAP,0.000459,1.193749,-0.193348,-7.530446
164,OXY,-0.000956,1.036652,-0.068460,-3.249811
165,QRVO,0.000130,1.355739,-0.103351,-3.303907
166,LH,0.000246,0.906864,-0.159404,-4.401495
